In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import re
import time
import warnings
import matplotlib.pyplot as plt

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2244: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2244: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_electricity.py'>

In [5]:
##  IMPORT SOME ATTRIBUTES, MODELS, AND SHARED VARIABLES

attr_fuel = sa.model_attributes.get_attribute_table(f"{sa.model_attributes.subsec_name_enfu}")
attr_region = sa.model_attributes.dict_attributes.get(f"{sa.model_attributes.dim_region}")
attr_technology = sa.model_attributes.get_attribute_table(f"{sa.model_attributes.subsec_name_entc}")
attr_time_period = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_time_period}")
attr_time_slice = sa.model_attributes.dict_attributes.get(f"time_slice")

# map each country to ISO code 3 and each code to 
dict_country_to_iso = dict((k, v.upper()) for k, v in attr_region.field_maps.get(f"{attr_region.key}_to_iso_alpha_3").items())
dict_iso_to_country = sf.reverse_dict(dict_country_to_iso)
dict_country_to_cf_region = attr_region.field_maps.get(f"{attr_region.key}_to_hourly_capacity_factor_region")
dict_iso_to_cf_region = dict((dict_country_to_iso.get(k), v) for k, v in dict_country_to_cf_region.items())
all_iso = list(dict_iso_to_country.keys())

# some time period oriented tools
dict_year_to_time_period = attr_time_period.field_maps.get(f"year_to_{attr_time_period.key}")
def year_to_tp(
    year: int
) -> int:

    m = min((dict_year_to_time_period.keys()))
    m_v = dict_year_to_time_period.get(m)

    diff = m - m_v

    return dict_year_to_time_period.get(year, year - diff)


# reformat simulations
field_country = "Country"
field_date_string = "date_string"
field_generation = "generation_gwh"
field_gwp = "max_generation_gwp"
field_iso = "iso_code3"
field_key = "GHD_ID"
field_month = "month"
field_ndays = "n_days"
field_technology = "technology"
field_year = "year"

# call variables from the electric model
model_elec = ml.ElectricEnergy(sa.model_attributes, sa.dir_jl, sa.dir_ref_nemo, initialize_julia = False)
model_energy = me.NonElectricEnergy(sa.model_attributes)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2244: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py:1274: UserWarning: Warning passed from optional_log: Successfully read NemoMod input table data from /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/nemo_mod/CapacityFactor.csv.
  warnings.warn(f"Warning passed from optional_log: {msg}.")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py:1274: UserWarning: Warning passed from optional_log: Successfully read NemoMod input table data from /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/nemo_mod/SpecifiedDemandPr

# Notebook Contents

This notebook inlcudes several basic datasets:
- Basic enery costs (not most current)
- Residual Capacities and assumed technology lifetimes
- Baseline Minimum Production Shares (MinShareProduction)
- Electricity Transmission Loss

##  Energy Costs from EIA
- source: https://www.eia.gov/outlooks/aeo/assumptions/pdf/table_8.2.pdf

In [37]:

fp_read = "/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/data_tables_and_derivations/ENERGY/eia_outlooks_table_8.2.xlsx"
df_eia = pd.read_excel(fp_read, skiprows = 1).rename(columns = {
    "Unnamed: 0": "tech", 
    "Unnamed: 1": "year_start",
    "Total Overnight Cost (2021$/kW)": "capital_cost",
    "Variable O&M (2021 $/MWh)": "variable_cost", 
    "Fixed O&M 2021$/kW-y)": "fixed_cost"
})

fields_group = ["cat_technology"]
fields_mean = [f"{x}_cost" for x in ["capital", "variable", "fixed"]]

dict_agg = dict(zip(fields_group, ["first" for x in fields_group]))
dict_agg.update(dict(zip(fields_mean, ["mean" for x in fields_mean])))

df_eia = df_eia.groupby(fields_group).agg(dict_agg).reset_index(drop = True).sort_values(by = ["cat_technology"])
df_eia.to_csv("/Users/jsyme/Desktop/tmp.csv", index = None, encoding = "UTF-8")

##  Build Residual Capacity Data
- See inline source commenting
- WRI Global Power Plant database: https://datasets.wri.org/dataset/globalpowerplantdatabase
- powerplant level data may be incomplete, so scale to aggregate statistics from UN http://data.un.org/Data.aspx?d=EDATA&f=cmID%3AEC

In [3]:
########################################################
###                                                  ###
###    BUILD NEMOMOD ReserveCapacity INITIAL DATA    ###
###                                                  ###
########################################################

# get data 
fp_data = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/LAC_global_power_plant_database.csv"
df_data = pd.read_csv(fp_data)
if False:
    df_data.dropna(
        how = "all", 
        subset = ["estimated_generation_gwh_2017", "estimated_generation_gwh_2016", "estimated_generation_gwh_2015", "estimated_generation_gwh_2014", "estimated_generation_gwh_2013"],
        inplace = True
    )

# assumed lifetimes (baseline) - add sources to attribute table
dict_lifetimes = {
    "Biomass": 25, # https://www.nrel.gov/analysis/tech-footprint.html
    "Other": 50, 
    "Gas": 25, # 22, but set to 25 https://www.eia.gov/todayinenergy/detail.php?id=34172
    "Hydro": 100, # https://www.nrel.gov/docs/fy04osti/34916.pdf
    "Oil": 40, 
    "Nuclear": 30, # https://www.iaea.org/sites/default/files/29402043133.pdf
    "Coal": 50, # https://www.nature.com/articles/s41467-019-12618-3
    "Solar": 30, # https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiExIeGvL35AhVPKkQIHR1ABJMQFnoECBsQAw&url=https%3A%2F%2Fnews.energysage.com%2Fhow-long-do-solar-panels-last%2F&usg=AOvVaw0rJ8w3zaBIP4b83sJgsKcr
    "Wind": 20, # https://nepis.epa.gov/Exe/ZyNET.exe/P100IL8K.TXT?ZyActionD=ZyDocument&Client=EPA&Index=2011+Thru+2015&Docs=&Query=&Time=&EndTime=&SearchMethod=1&TocRestrict=n&Toc=&TocEntry=&QField=&QFieldYear=&QFieldMonth=&QFieldDay=&IntQFieldOp=0&ExtQFieldOp=0&XmlQuery=&File=D%3A%5Czyfiles%5CIndex%20Data%5C11thru15%5CTxt%5C00000010%5CP100IL8K.txt&User=ANONYMOUS&Password=anonymous&SortMethod=h%7C-&MaximumDocuments=1&FuzzyDegree=0&ImageQuality=r75g8/r75g8/x150y150g16/i425&Display=hpfr&DefSeekPage=x&SearchBack=ZyActionL&Back=ZyActionS&BackDesc=Results%20page&MaximumPages=1&ZyEntry=1&SeekPage=x&ZyPURL
    "Waste": 30, # https://www.pbs.org/newshour/science/is-burning-trash-a-good-way-to-dispose-of-it-waste-incineration-in-charts,
    "Geothermal": 30, # https://geothermal-energy-journal.springeropen.com/articles/10.1186/s40517-021-00183-2
    "Ocean": 25 # assume 25, nascent tech
}

# real lifetimes are available here
attr_entc = sa.model_attributes.get_attribute_table(sa.model_attributes.subsec_name_entc)
dict_lifetimes = attr_entc.field_maps.get("cat_technology_to_operational_life")


# FOR PURPOSES OF INITIAL STATES, SET PETCOKE TO COAL
if len(df_data[df_data["primary_fuel"] == "Petcoke"]) == 1:
    df_data["primary_fuel"] = df_data["primary_fuel"].replace({"Petcoke": "Coal"})
    
# FOR OTHER POWER PLANTS, USE SECONDARY
if len(df_data["primary_fuel"][df_data["primary_fuel"] == "Other"]) > 0:
    inds = df_data[df_data["primary_fuel"] == "Other"].index
    
    for i in inds:
        if df_data["other_fuel1"].iloc[i] not in [np.nan, "Other", "Petcoke"]:
            df_data["primary_fuel"].iloc[i] = df_data["other_fuel1"].iloc[i]
        elif df_data["name"].iloc[i] == "Sol":
            df_data["primary_fuel"].iloc[i] = "Solar"
            
# CONVER TO FORMAT COMPATIBLE WITH SISEPUEDE
df_data["primary_fuel"] = df_data["primary_fuel"].replace(
    {
        "Hydro": "Hydropower",
        "Waste": "Waste Incineration"
    }
)

all_fuel = list(set(df_data["primary_fuel"]))
dict_repl_fuel = {}
for fuel in all_fuel:
    fuel_new = fuel.lower().replace(" ", "_")
    fuel_new = f"pp_{fuel_new}"
    dict_repl_fuel.update({fuel: fuel_new})
df_data["primary_fuel"] = df_data["primary_fuel"].replace(dict_repl_fuel)
            
#############################################
#    FILL IN MISSING COMMISSIONING YEARS    #
#############################################

#
# TO FILL MISSING COMMISSION YEARS, GET MEAN COMMISSION YEAR FOR PLANTS BY TYPE IN LAC
# - USE RANDOM NUMBERS WITH A SEED
# - NEED TO ESTIMATE WHEN EXISTING PLANTS GO OFFLINE
# - CAN IMPROVE WITH BETTER INFORMATION LATER
#

all_plants = list(set(df_data["primary_fuel"]))
dict_mean_commission_year_by_plant = {}
dict_std_commission_year_by_plant = {}
for plant in all_plants:
    df_tmp = df_data[df_data["primary_fuel"] == plant]
    yr_mean_commission = np.array(df_tmp["commissioning_year"])
    yr_mean_commission = yr_mean_commission[np.where(~np.isnan(yr_mean_commission))[0]]
    yr_std_commission = np.std(yr_mean_commission)
    yr_mean_commission = int(np.round(np.mean(yr_mean_commission)))
    dict_mean_commission_year_by_plant.update({plant: yr_mean_commission})
    dict_std_commission_year_by_plant.update({plant: yr_std_commission})

# initialize some components
countries = list(set(df_data["country_long"]))
countries.sort()
df_years = pd.DataFrame({"year": range(1920, 2056)})

# set a seed - I just chose 50
np.random.seed(50)

df_out_total = []
for country in enumerate(countries):
    ind_country, country = country
    
    df_tmp = df_data[df_data["country_long"] == country].copy().reset_index(drop = True)
    # check commision years
    df_na_comissions = df_tmp[df_tmp["commissioning_year"].isna()]
    inds_na_commissions = df_na_comissions.index
    for i in enumerate(inds_na_commissions):
        i, ind = i
        plant = str(df_na_comissions["primary_fuel"].iloc[i])
        mu = dict_mean_commission_year_by_plant[plant]
        sd = dict_std_commission_year_by_plant[plant]
        rand_yr = int(min(np.random.normal(mu, sd), 2018))
        df_tmp["commissioning_year"].iloc[ind] = rand_yr

    df_years_tmp = []#df_years.copy()
    df_years_out = df_years.copy()
    
    for i in range(len(df_tmp)):
        field_plant = f"plant_{i}"
        plant = str(df_tmp["primary_fuel"].iloc[i])
        commission_year = int(df_tmp["commissioning_year"].iloc[i])
        lifetime = dict_lifetimes.get(plant)
        capacity = float(df_tmp["capacity_mw"].iloc[i])
        
        df_years_merge = pd.DataFrame({
            "year": range(commission_year, commission_year + lifetime), 
            "capacity": capacity,
            "plant": plant
        })
        
        if len(df_years_tmp) == 0:
            df_years_tmp = [df_years_merge for x in range(len(df_tmp))]
        else:
            df_years_tmp[i] = df_years_merge[df_years_tmp[0].columns]
            
    df_years_tmp = pd.concat(df_years_tmp, axis = 0)
    df_years_tmp = df_years_tmp.groupby(["year", "plant"]).agg({"year": "first", "plant": "first", "capacity": "sum"}).reset_index(drop = True)
    #
    df_years_out = pd.merge(df_years_out, df_years_tmp, how = "left")
    df_years_out["capacity"] = df_years_out["capacity"].fillna(0)
    df_years_out = df_years_out.dropna(how = "any", subset = ["plant"]).sort_values(by = ["year", "plant"]).reset_index(drop = True)
    df_years_out["country"] = country.lower().replace(" ", "_");
    
    df_years_out = pd.pivot(
        df_years_out,
        ["year", "country"], 
        ["plant"], 
        "capacity"
    ).reset_index()
    
    df_out = pd.DataFrame()
    for k in df_years_out.columns:
        df_out[k] = df_years_out[k].copy().fillna(0.0)
    
    if len(df_out_total) == 0:
        df_out_total = [df_out for x in countries]
    else:
        df_out_total[ind_country] = df_out
    
df_out_total = pd.concat(df_out_total, axis = 0).fillna(0)


##  FORMAT VARIABLES FOR INGESTION

model_elec = ml.ElectricEnergy(
    sa.model_attributes, 
    sa.dir_jl,
    sa.dir_ref_nemo,
    initialize_julia = False
)

fields_rnm = [x for x in attr_entc.key_values if x in df_out_total.columns]
fields_new = sa.model_attributes.build_varlist(
    "Energy Technology", model_elec.modvar_entc_nemomod_residual_capacity,
    restrict_to_category_values = fields_rnm
)
dict_rnm = dict(zip(fields_rnm, fields_new))

#
#  do units conversion
#

units_target = sa.model_attributes.get_variable_characteristic(
    model_elec.modvar_entc_nemomod_residual_capacity, 
    sa.model_attributes.varchar_str_unit_power
)
scalar = sa.model_attributes.get_power_equivalent("mw", units_target)

for field in fields_rnm:
    df_out_total[field] = np.array(df_out_total[field])*scalar


df_out_total.rename(columns = dict_rnm, inplace = True)
fields_ind = [x for x in ["year", "country"] if x in df_out_total.columns]
fields_dat = sorted([x for x in df_out_total.columns if (x not in fields_ind)])

df_out_total = df_out_total[fields_ind + fields_dat]



###############################################################
###                                                         ###
###    ADD UN AGGREGATE DATA TO SCALE UP WHERE NECESSARY    ###
###                                                         ###
###############################################################

field_total_capacity = f"total_capacity_{units_target}"
df_out_total[field_total_capacity] = df_out_total[fields_new].sum(axis = 1)


##  integrate aggreate production from UN data to scale up Residual Capacities 
df_un_pp_agg = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/UNdata_Export_20230307_234559434.csv")


# some basic fields
field_capacity = "capacity"
field_commodity = "Commodity - Transaction"
field_plant_type = "plant_type_un"

# map power plants to grouping in UN
dict_plant_to_subgroup = {
    "pp_biogas": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_biomass": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_coal": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_coal_ccs": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_geothermal": "Electricity - total net installed capacity of electric power plants, geothermal",
    "pp_hydropower": "Electricity - total net installed capacity of electric power plants, hydro",
    "pp_gas": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_gas_ccs": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_nuclear": "Electricity - total net installed capacity of electric power plants, nuclear",
    "pp_ocean": "Electricity - total net installed capacity of electric power plants, tide, wave, marine",
    "pp_oil": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_solar": "Electricity - total net installed capacity of electric power plants, solar",
    "pp_waste_incineration": "Electricity - total net installed capacity of electric power plants, combustible fuels",
    "pp_wind": "Electricity - total net installed capacity of electric power plants, wind"
}

# reduce to scale to aggregate installed capacity
df_capacity_un_total = df_un_pp_agg[
    df_un_pp_agg["Commodity - Transaction"].isin(
        [
            "Electricity - total net installed capacity of electric power plants, main activity & autoproducer"
        ]
    )
]

df_capacity_un_by_subgroup = df_un_pp_agg[
    df_un_pp_agg["Commodity - Transaction"].isin(
        dict_plant_to_subgroup.values()
    )
]



##  CLEAN THE DATA

# rename
dict_rnm = {
    "Country or Area": attr_region.key,
    "Year": field_year,
    "Quantity": field_capacity,
    field_commodity: field_plant_type
}
df_capacity_un_total = df_capacity_un_total[dict_rnm.keys()].rename(columns = dict_rnm).drop(field_plant_type, axis = 1)
df_capacity_un_by_subgroup = df_capacity_un_by_subgroup[dict_rnm.keys()].rename(columns = dict_rnm)


# UN data are in mw
units_target = sa.model_attributes.get_variable_characteristic(
    model_elec.modvar_entc_nemomod_residual_capacity, 
    sa.model_attributes.varchar_str_unit_power
)
scalar = sa.model_attributes.get_power_equivalent("mw", units_target)
# standardize time period, region, year
dict_region_repl = {
    "bolivia_(plur._state_of)": "bolivia",
    "venezuela_(bolivar._rep.)": "venezuela"
}
# standardize index fields - subgroups
df_capacity_un_by_subgroup[attr_region.key] = [x.lower().replace(" ", "_") for x in list(df_capacity_un_by_subgroup[attr_region.key])]
df_capacity_un_by_subgroup[attr_region.key].replace(dict_region_repl, inplace = True)
df_capacity_un_by_subgroup[field_year] = np.array(df_capacity_un_by_subgroup[field_year]).astype(int)
df_capacity_un_by_subgroup[attr_time_period.key] = df_capacity_un_by_subgroup[field_year].apply(year_to_tp)
df_capacity_un_by_subgroup[field_capacity] = np.array(df_capacity_un_by_subgroup[field_capacity])*scalar
# standardize index fields - total
df_capacity_un_total[attr_region.key] = [x.lower().replace(" ", "_") for x in list(df_capacity_un_total[attr_region.key])]
df_capacity_un_total[attr_region.key].replace(dict_region_repl, inplace = True)
df_capacity_un_total[field_year] = np.array(df_capacity_un_total[field_year]).astype(int)
df_capacity_un_total[attr_time_period.key] = df_capacity_un_total[field_year].apply(year_to_tp)
df_capacity_un_total[field_capacity] = np.array(df_capacity_un_total[field_capacity])*scalar


##  BUILD SCALARS

field_scale_residual_capacity = "scalar_residual_capacity"
df_get_scalars = pd.merge(
    df_out_total[[field_year, field_country.lower(), field_total_capacity]],
    df_capacity_un_total.rename(
        columns = {
            field_capacity: f"{field_capacity}_un",
            attr_region.key: field_country.lower()
        }
    ),
    how = "left"
).dropna()

df_get_scalars[field_scale_residual_capacity] = sf.vec_bounds(
    np.nan_to_num(
        np.array(df_get_scalars[f"{field_capacity}_un"])/np.array(df_get_scalars[field_total_capacity]),
        0.0,
        posinf = 0.0
    ),
    (1, np.inf)
)


# get scalars by region to convert power plants to aggregate metrics from UN
df_scalars_by_region = df_get_scalars.groupby([field_country.lower()])
df_scalars_by_region_cln = []
df_left = pd.DataFrame({field_year: list(range(min(df_get_scalars["year"]), max(df_out_total[field_year]) + 1))})
for i, df in df_scalars_by_region:
    # get last residual capacity scalar
    df = df.sort_values(by = [field_year], ascending = False)
    scalar_final = float(df[field_scale_residual_capacity].iloc[0])

    df = pd.merge(
        df_left, 
        df[[field_year, field_country.lower(), field_scale_residual_capacity]], 
        how = "left"
    )

    df[field_country.lower()] = i
    df[field_scale_residual_capacity].interpolate(method = "pad", inplace = True)
    df[field_scale_residual_capacity].interpolate(method = "bfill", inplace = True)
    
    df_scalars_by_region_cln.append(df)

df_scalars_by_region = pd.concat(df_scalars_by_region_cln, axis = 0).reset_index(drop = True)



# MERGE INTO DF_OUT_TOTAL

df_out_total_rescaled = pd.merge(
    df_out_total,
    df_scalars_by_region,
    how = "left"
)
df_out_total_rescaled[field_scale_residual_capacity].interpolate(method = "bfill", inplace = True)

# finally, provide rescale
for fld in fields_new:
    df_out_total_rescaled[fld] = np.array(df_out_total_rescaled[fld])*np.array(df_out_total_rescaled[field_scale_residual_capacity])

    
    
    

if True:
    df_out_total_rescaled.drop(
        [field_total_capacity, field_scale_residual_capacity], 
        axis = 1
    ).to_csv(
        sa.fp_csv_nemomod_residual_capacity_inputs, 
        index = None,
        encoding = "UTF-8"
    )
    
"""
regex_gen_capacity = re.compile("generation_gwh_(\d*$)")
fields_gen_capac = [x for x in df_data.columns if regex_gen_capacity.match(x) is not None]

df_data.dropna(subset = fields_gen_capac, how = "all")["country"].unique()
#df_data.columns
""";

/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_12363/4267709892.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data["primary_fuel"].iloc[i] = df_data["other_fuel1"].iloc[i]
/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_12363/4267709892.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data["primary_fuel"].iloc[i] = "Solar"
/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_12363/4267709892.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

#  Build MinShareProduction data 
- Currently read in aggregate, based on Monthly Data from IEA
- Used to ensure historical production aligns

**NOTE** will require integrating additional code to aggregate monthly data
- Based on IEA monthly electricity generation data
- See https://www.iea.org/data-and-statistics/data-product/monthly-electricity-statistics#monthly-electricity-statistics


In [68]:
# read in data
dfs_production_by_country = pd.read_excel(
    "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/historical_electricity_production_fractions_for_James_2.23.xlsx",
    sheet_name = None
)

field_msp = "fraction_production"


##  retrieve and clean

df_production_by_country = dfs_production_by_country.get(
    "Per Country and Year"
).drop(
    ["Alpha-2 code"],
    axis = 1
)
df_avg_production_by_country = dfs_production_by_country.get(
    "Per Country Averaged Across Yrs"
).drop(
    ["Alpha-2 code"],
    axis = 1
)
df_avg_production_lac = dfs_production_by_country.get(
    "LAC Average"
)[
    [field_technology, "fraction_production_weighted_by_production"]
].rename(columns = {"fraction_production_weighted_by_production": field_msp})


# clean fields
df_production_by_country[field_iso] = [
    dict_country_to_iso.get(x.lower().replace(" ", "_")) 
    for x in list(df_production_by_country[field_country])
]
df_avg_production_by_country[field_iso] = [
    dict_country_to_iso.get(x.lower().replace(" ", "_")) 
    for x in list(df_avg_production_by_country[field_country])
]



df_production_by_country.drop([field_country], axis = 1, inplace = True)
df_avg_production_by_country.drop([field_country], axis = 1, inplace = True)


##  drop integrated techs for now

cats_entc_drop = ["pp_waste_incineration", "pp_biogas"]
df_production_by_country = df_production_by_country[
    ~df_production_by_country[field_technology].isin(cats_entc_drop)
].reset_index(drop = True)

df_avg_production_by_country = df_avg_production_by_country[
    ~df_avg_production_by_country[field_technology].isin(cats_entc_drop)
].reset_index(drop = True)

df_avg_production_lac = df_avg_production_lac[
    ~df_avg_production_lac[field_technology].isin(cats_entc_drop)
].reset_index(drop = True)


# replace 0s in years in observations -- also, set sets of tech & isos availabl

all_technology = sorted(list(df_production_by_country[field_technology].unique()))
all_iso_defined_in_production = sorted(list(df_production_by_country[field_iso].unique()))

years_merge = range(
    min(df_production_by_country["Year"]), 
    max(df_production_by_country["Year"]) + 1
)
df_left = pd.DataFrame({field_year: years_merge})
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_technology: all_technology})
)
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_iso: all_iso_defined_in_production})
)

# merge to all years/techs available
df_production_by_country = pd.merge(
    df_left, 
    df_production_by_country.rename(columns = {"Year": field_year}),
    how = "left"
)
df_production_by_country[field_msp].fillna(0.0, inplace = True)
# clean the time period
df_production_by_country[sa.model_attributes.dim_time_period] = df_production_by_country[
    field_year
].apply(year_to_tp).astype(int)
df_long = df_production_by_country.copy()


##  UPDATE TIME PERIOD

##  group by country and pivot
 
df_production_by_country = sf.pivot_df_clean(
    df_production_by_country,
    [field_technology],
    [field_msp]
)

df_out = [
    df_production_by_country
]



##  NEXT, EXPAND TO ALL YEARS

years_merge = range(
    max(df_production_by_country[field_year]) + 1, 
    max(attr_time_period.table[field_year]) + 1
)
df_left = pd.DataFrame({field_year: years_merge})
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_technology: all_technology})
)
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_iso: all_iso_defined_in_production})
)

# use averages for all future dates 
df_production_by_country_append = pd.merge(
    df_left,
    df_avg_production_by_country,
    how = "left"
).fillna(0.0)
df_production_by_country_append = sf.pivot_df_clean(
    df_production_by_country_append,
    [field_technology],
    [field_msp]
)

# clean the time period
df_production_by_country_append[sa.model_attributes.dim_time_period] = df_production_by_country_append[
    field_year
].apply(year_to_tp).astype(int)

df_out += [
    df_production_by_country_append
]






###  FINALLY, USE AVERAGES FOR OTHER COUNTRIES

years_merge = range(
    min(df_production_by_country[field_year]), 
    max(attr_time_period.table[field_year]) + 1
)
df_left = pd.DataFrame({field_year: years_merge})
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_technology: all_technology})
)
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_iso: [x for x in all_iso if x not in all_iso_defined_in_production]})
)

# merge to all years/regions that are missing

df_avg_production_lac = pd.merge(
    df_left,
    df_avg_production_lac,
    how = "outer"
).fillna(0.0)

df_avg_production_lac = sf.pivot_df_clean(
    df_avg_production_lac,
    [field_technology],
    [field_msp]
)

# clean the time period
df_avg_production_lac[sa.model_attributes.dim_time_period] = df_avg_production_lac[
    field_year
].apply(year_to_tp).astype(int)

df_out += [df_avg_production_lac]


##  FORMAT OUTPUT DATASET

df_out = pd.concat(df_out, axis = 0).sort_values(by = [field_iso, field_year]).reset_index(drop = True)

fields_group = [field_year, attr_time_period.key, field_iso]
fields_data = [x for x in attr_technology.key_values if x in df_out.columns]

# name as MSP variable
modvar = model_elec.modvar_entc_nemomod_min_share_production
subsec = model_elec.model_attributes.get_variable_subsector(modvar)
fields_new = sa.model_attributes.build_varlist(
    subsec, 
    modvar,
    restrict_to_category_values = fields_data
)

dict_rnm = dict(zip(fields_data, fields_new))


df_out_grouped = df_out.groupby([field_iso])
df_out_new = []
dfk = None
for i, df in df_out_grouped:
    
    yrs = df[fields_group].copy()
    
    df["TMP"] = df[fields_data].sum(axis = 1)
    
    df = pd.merge(
        yrs,
        df[df["TMP"] > 0],
        how = "left"
    )
    dfk = df if (i == "CRI") else dfk
    
    # perform interpolations
    df[fields_data] = df[fields_data].interpolate()
    df[fields_data] = df[fields_data].interpolate(method = "bfill")
    
    df_out_new.append(df)

    
df_out = pd.concat(df_out_new, axis = 0).reset_index(drop = True)
df_out = df_out[fields_group + fields_data].rename(columns = dict_rnm)
df_out.to_csv(
    sa.fp_csv_nemomod_minimum_share_of_production_baselines,
    index = None,
    encoding = "UTF-8"
)

#if False:
    
    

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py:1347: FutureWarning: In a future version of pandas all arguments of pivot except for the argument 'data' will be keyword-only.
  df_piv = pd.pivot(
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py:1347: FutureWarning: In a future version of pandas all arguments of pivot except for the argument 'data' will be keyword-only.
  df_piv = pd.pivot(
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py:1347: FutureWarning: In a future version of pandas all arguments of pivot except for the argument 'data' will be keyword-only.
  df_piv = pd.pivot(
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:3752: UserWarning: clean_partial_category_dictionary: Invalid categories values 'fp_hydrogen_electrolysis', 'fp_hydrogen_gasification', 'fp_hydrogen_reformation', 'pp_biogas', 'pp_biomass', 'pp_coal_cc

In [65]:
df_out[df_out[field_iso].isin(["CRI"])]



,year,time_period,iso_code3,nemomod_entc_frac_min_share_production_pp_coal,nemomod_entc_frac_min_share_production_pp_gas,nemomod_entc_frac_min_share_production_pp_geothermal,nemomod_entc_frac_min_share_production_pp_hydropower,nemomod_entc_frac_min_share_production_pp_nuclear,nemomod_entc_frac_min_share_production_pp_oil,nemomod_entc_frac_min_share_production_pp_solar,nemomod_entc_frac_min_share_production_pp_wind
328,2010,-5,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
329,2011,-4,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
330,2012,-3,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
331,2013,-2,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
332,2014,-1,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
333,2015,0,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
334,2016,1,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
335,2017,2,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
336,2018,3,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824
337,2019,4,CRI,0.0,0.0,0.117165,0.734915,0.0,0.000229,0.006583,0.125824


# Get Electric Transmission Loss data
- source of CSV (World Bank): https://data.worldbank.org/indicator/EG.ELC.LOSS.ZS

In [330]:
df_losses = pd.read_csv(
    "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/API_EG.ELC.LOSS.ZS_DS2_en_csv_v2_4898900/API_EG.ELC.LOSS.ZS_DS2_en_csv_v2_4898900.csv",
    skiprows = 3
)

# some filtering
field_cc = "Country Code"
indicator = "Electric power transmission and distribution losses (% of output)"
df_losses = df_losses[
    df_losses["Indicator Name"] == indicator
].reset_index(drop = True).rename(columns = {field_cc: field_iso}).dropna(how = "all", axis = 1)

# get variable name
subsec = sa.model_attributes.get_variable_subsector(
    model_elec.modvar_enfu_transmission_loss_frac_electricity
)
field_var = sa.model_attributes.build_varlist(
    subsec,
    model_elec.modvar_enfu_transmission_loss_frac_electricity,
    restrict_to_category_values = [model_elec.cat_enfu_elec]
)[0]


# get data and melt
fields_data = [x for x in df_losses.columns if str(x).isnumeric()]
df_losses = pd.melt(
    df_losses[[field_iso] + fields_data],
    [field_iso],
    fields_data,
    value_name = field_var,
    var_name = field_year
).dropna().reset_index(drop = True)
# convert strings to int
df_losses[field_year] = [int(x) for x in list(df_losses[field_year])]

# get full set of years to merge into 
year_min = 2010
years_merge = range(
    year_min, 
    max(dict_year_to_time_period.keys()) + 1
)
df_left = pd.DataFrame({field_year: years_merge})
df_left = sf.explode_merge(
    df_left,
    pd.DataFrame({field_iso: all_iso})
)

# group and iterate
dfs_losses = df_losses.groupby([field_iso])
df_out = []
for i, df in dfs_losses:
    
    if i in all_iso:
        df_cur = df.sort_values(by = ["year"], ascending = False)
        mv = np.mean(np.array(df_cur[field_var])[0:min(5, len(df))])

        df_cur = pd.merge(df_left[df_left[field_iso] == i], df_cur, how = "left")
        df_cur = df_cur[df_cur[field_year] >= year_min].fillna(mv)
        
        # add time period and reduce to fraction
        df_cur[attr_time_period.key] = df_cur[field_year].apply(year_to_tp)
        df_cur[field_var] = sf.vec_bounds(np.array(df_cur[field_var])/100, (0.0, 1.0))
        
        df_out.append(df_cur)
        
fields_ord = [field_iso, field_year, attr_time_period.key, field_var]
df_out = pd.concat(df_out, axis = 0)[fields_ord].sort_values(by = [field_iso, field_year]).reset_index(drop = True)

df_out.to_csv(
    sa.fp_csv_nemomod_transmission_losses,
    index = None,
    encoding = "UTF-8"
)

/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_2434/2431838223.py:51: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in dfs_losses:


##  FUEL COSTS
- fuel costs come from Edmundo
- tonne/barrel of oil comes from https://sciencing.com/convert-metric-tons-barrels-8220711.html
    - 0.14459225
- m3/barrel comes from https://www.metric-conversions.org
    - 0.158987

In [213]:
df_fuel = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/fuel_prices_from_edmundo_20230306_with_thermal_hydrogen.csv")

warnings.filterwarnings("ignore")

# set some field names
field_fuel = "fuel"
field_price = "price"
field_unit = "UNIT"
field_unit_denominator = "unit_denominator"
field_tmp_scalar = "scalar"

# some other fuel conversion costs
oil_tonne_per_barrel = 0.14459225
oil_m3_per_barrel = 0.158987
oil_tonne_per_m3 = oil_tonne_per_barrel/oil_m3_per_barrel

# drop some fields
fields_drop = [x for x in df_fuel.columns if x in ["Unnamed: 0", "Country.Name", "unit_type"]]
df_fuel.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None

# expand fuel before filling NAs
df_fuel_all = sf.explode_merge(
    df_fuel[[field_fuel, field_unit_denominator]].drop_duplicates(),
    df_fuel[[field_iso]].drop_duplicates()
)
df_fuel = pd.merge(
    df_fuel_all,
    df_fuel, 
    how = "left"
)


# loop over fuels and replace oil units if necessary
df_fuel_grouped = df_fuel.groupby([field_fuel])
fuels_mass_to_volume = ["fuel_oil", "fuel_crude"]

df_fuel_new = []

for i, df in df_fuel_grouped:
    if i in fuels_mass_to_volume:
        if set(df[field_unit_denominator]) == set({"tonne"}):
            df[field_unit_denominator].replace({"tonne": "m3"}, inplace = True)
            df[field_price] = np.array(df[field_price])*oil_tonne_per_m3
        else:
            print(f"Check fuel {fuel}: not entered in tonnes")

    # get regional mean
    price_mean = np.array(list(set(
        df[
            df[field_iso].isin(dict_iso_to_country.keys()) & 
            ~df[field_price].isna()
        ][field_price]
    ))).mean()

    df[field_price].fillna(price_mean, inplace = True)

    df_fuel_new.append(df)

df_fuel = pd.concat(df_fuel_new, axis = 0).reset_index(drop = True)
df_fuelc = df_fuel.copy()


# replace input units
dict_repl_units = {
    'Mwh': "mmbtu", 
    "mwh": "mmbtu",
    'liter': "m3", 
    'tonne': "tonne", 
    'MWH': "mmbtu", 
    '1000 liters': "m3", 
    '1000 liter': "m3", 
    'MBtu': "mmbtu"
}

dict_repl_num_units = {
    "Total price (USD/unit using PPP)": 1,
    "Total price (USD/unit)": 1
}

# add scalars in terms of old per new

# get variable units
units_gravimetric = sa.model_attributes.get_variable_characteristic(
    model_energy.modvar_enfu_price_gravimetric,
    sa.model_attributes.varchar_str_unit_mass
)
units_thermal = sa.model_attributes.get_variable_characteristic(
    model_energy.modvar_enfu_price_thermal,
    sa.model_attributes.varchar_str_unit_energy
)
units_volumetric = sa.model_attributes.get_variable_characteristic(
    model_energy.modvar_enfu_price_volumetric,
    sa.model_attributes.varchar_str_unit_volume
)

dict_repl_units_scalars = {
    'Mwh': sa.model_attributes.get_energy_equivalent("mwh", units_thermal), 
    'liter': sa.model_attributes.get_volume_equivalent("litre", units_volumetric), 
    'tonne': sa.model_attributes.get_mass_equivalent("tonne", units_gravimetric), 
    'MWH': sa.model_attributes.get_energy_equivalent("mwh", units_thermal),
    '1000 liters': sa.model_attributes.get_volume_equivalent("m3", units_volumetric),
    '1000 liter': sa.model_attributes.get_volume_equivalent("m3", units_volumetric), 
    "m3": sa.model_attributes.get_volume_equivalent("m3", units_volumetric),
    'MBtu': sa.model_attributes.get_energy_equivalent("mbtu", units_thermal),
}

#
df_fuel[field_tmp_scalar] = df_fuel[field_unit_denominator].replace(dict_repl_units_scalars)
df_fuel[field_price] = np.array(df_fuel[field_price])/np.array(df_fuel[field_tmp_scalar])
df_fuel[field_unit_denominator].replace(dict_repl_units, inplace = True)



# loop over fuels again, group to build variable 
df_fuel_grouped = df_fuel.groupby([field_fuel])

cats_mass = sa.model_attributes.get_variable_categories(model_energy.modvar_enfu_price_gravimetric)
cats_thermal = sa.model_attributes.get_variable_categories(model_energy.modvar_enfu_price_thermal)
cats_volume = sa.model_attributes.get_variable_categories(model_energy.modvar_enfu_price_volumetric)

# should all be the same, but just for consistency's sake
subsec_mass = sa.model_attributes.get_variable_subsector(model_energy.modvar_enfu_price_gravimetric)
subsec_thermal = sa.model_attributes.get_variable_subsector(model_energy.modvar_enfu_price_thermal)
subsec_volume = sa.model_attributes.get_variable_subsector(model_energy.modvar_enfu_price_volumetric)

dict_repl = {}
fuels_unresolved = []


for i, df in df_fuel_grouped:
    
    new_val = None
    
    if (str(df[field_unit_denominator].iloc[0]) == units_gravimetric) & (i in cats_mass):
        
        # check mass
        new_val = sa.model_attributes.build_varlist(
            subsec_mass,
            model_energy.modvar_enfu_price_gravimetric,
            restrict_to_category_values = [i]
        )[0]
    
    if (str(df[field_unit_denominator].iloc[0]) == units_thermal) & (i in cats_thermal):
        
        # check thermal
        new_val = sa.model_attributes.build_varlist(
            subsec_thermal,
            model_energy.modvar_enfu_price_thermal,
            restrict_to_category_values = [i]
        )[0]
    
    if (str(df[field_unit_denominator].iloc[0]) == units_volumetric) & (i in cats_volume):
        
        # check volume
        new_val = sa.model_attributes.build_varlist(
            subsec_volume,
            model_energy.modvar_enfu_price_volumetric,
            restrict_to_category_values = [i]
        )[0]
        
    
    fuels_unresolved += [i] if (new_val is None) else []
    dict_repl.update({i: new_val}) if (new_val is not None) else None
    
        
df_fuel[field_fuel].replace(dict_repl, inplace = True)
    
# pivot and reorder
df_fuel = sf.pivot_df_clean(
    df_fuel[[field_iso, field_fuel, field_price]],
    [field_fuel],
    [field_price]
)

df_fuel = sf.explode_merge(
    attr_time_period.table[[attr_time_period.key]],
    df_fuel[
        df_fuel[field_iso].isin(dict_iso_to_country.keys())
    ]
).sort_values(by = [field_iso, attr_time_period.key]).reset_index(drop = True)
    

fields_index = [field_iso, attr_time_period.key]
df_fuel = df_fuel[fields_index + sorted([x for x in df_fuel.columns if x not in fields_index])]

df_fuel.to_csv(
    sa.fp_csv_nemomod_fuel_costs,
    index = None, 
    encoding = "UTF-8"
)



In [382]:
len(set(df_capacity_un_total["region"]) & set(df_out_total["country"]))
set(df_out_total["country"]) - set(df_capacity_un_total["region"])

{'bolivia', 'venezuela'}

{'bolivia_(plur._state_of)': 'bolivia',
 'venezuela_(bolivar._rep.)': 'venezuela'}

In [462]:
df_out_total_rescaled

,year,country,nemomod_entc_residual_capacity_pp_biomass_gw,nemomod_entc_residual_capacity_pp_coal_gw,nemomod_entc_residual_capacity_pp_gas_gw,nemomod_entc_residual_capacity_pp_geothermal_gw,nemomod_entc_residual_capacity_pp_hydropower_gw,nemomod_entc_residual_capacity_pp_nuclear_gw,nemomod_entc_residual_capacity_pp_oil_gw,nemomod_entc_residual_capacity_pp_solar_gw,nemomod_entc_residual_capacity_pp_waste_incineration_gw,nemomod_entc_residual_capacity_pp_wind_gw,total_capacity_gw,scalar_residual_capacity
0,1943,argentina,0.0,0.0,0.0,0.0,0.011797,0.0,0.005518,0.0,0.0,0.0,0.015852,1.092269
1,1944,argentina,0.0,0.0,0.0,0.0,0.040196,0.0,0.005518,0.0,0.0,0.0,0.041852,1.092269
2,1945,argentina,0.0,0.0,0.0,0.0,0.040196,0.0,0.005518,0.0,0.0,0.0,0.041852,1.092269
3,1946,argentina,0.0,0.0,0.0,0.0,0.040196,0.0,0.005518,0.0,0.0,0.0,0.041852,1.092269
4,1947,argentina,0.0,0.0,0.0,0.0,0.040196,0.0,0.005518,0.0,0.0,0.0,0.041852,1.092269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,2051,venezuela,0.0,0.0,0.0,0.0,23.064191,0.0,0.000000,0.0,0.0,0.0,17.666000,1.305569
2072,2052,venezuela,0.0,0.0,0.0,0.0,23.064191,0.0,0.000000,0.0,0.0,0.0,17.666000,1.305569
2073,2053,venezuela,0.0,0.0,0.0,0.0,23.064191,0.0,0.000000,0.0,0.0,0.0,17.666000,1.305569
2074,2054,venezuela,0.0,0.0,0.0,0.0,23.064191,0.0,0.000000,0.0,0.0,0.0,17.666000,1.305569


In [3]:
df_fuel = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/fuel_prices_from_edmundo_20230306.csv")


In [5]:
df_fuel[
    df_fuel["Country.Name"].isin(["Mexico"])&
    df_fuel["fuel"].isin(["fuel_hydrogen"])
]

,Unnamed: 0,Country.Name,fuel,UNIT,price,unit_denominator,unit_type,iso_code3
3209,3210,Mexico,fuel_hydrogen,Total price (USD/unit using PPP),4.445625,liter,volume,MEX


In [359]:
df_agg = df_capacity_un_by_subgroup.drop([field_plant_type, attr_time_period.key], axis = 1).groupby(["region", "year"]).agg(
    {"region": "first", "year": "first", "capacity": "sum"}
).reset_index(drop = True)

df_agg[df_agg["year"] == 2020]


,region,year,capacity
30,afghanistan,2020,537.000
62,albania,2020,2401.000
93,algeria,2020,21653.000
124,american_samoa,2020,47.066
156,andorra,2020,53.927
...,...,...,...
6897,viet_nam,2020,78066.000
6929,wallis_and_futuna_is.,2020,8.879
6960,yemen,2020,1771.800
6995,zambia,2020,3045.630


In [371]:
test = pd.merge(
    df_agg[df_agg["year"] == 2020],
    df_capacity_un_total.rename(columns = {"capacity": "cap_tot"}).drop(["time_period"], axis = 1),
    how = "left"
)

test["sort"] = np.array(test["capacity"]) - np.array(test["cap_tot"])


test.sort_values(by = ["sort"], ascending = True)

,region,year,capacity,cap_tot,sort
94,india,2020,443320.000,458430.000,-1.511000e+04
132,mexico,2020,91247.000,95045.000,-3.798000e+03
103,japan,2020,351347.327,353667.327,-2.320000e+03
214,united_states,2020,1146528.200,1147891.900,-1.363700e+03
217,uzbekistan,2020,16651.860,17501.860,-8.500000e+02
...,...,...,...,...,...
164,puerto_rico,2020,6125.605,6125.605,9.094947e-13
161,philippines,2020,26899.000,26899.000,3.637979e-12
199,thailand,2020,49550.340,49550.340,7.275958e-12
11,australia,2020,86711.048,86711.048,1.455192e-11


,region,year,capacity_un,time_period
0,afghanistan,2021,537.0,6
1,afghanistan,2020,537.0,5
2,afghanistan,2019,531.0,4
3,afghanistan,2018,519.0,3
4,afghanistan,2017,519.0,2
...,...,...,...,...
67985,zimbabwe,1994,1948.0,-21
67986,zimbabwe,1993,2078.0,-22
67987,zimbabwe,1992,2038.0,-23
67988,zimbabwe,1991,2038.0,-24
